# Chat with Video

In this example we're going to work through building a "chat-with-video" AI pipeline. We'll see how to:

1. Take any YouTube video and transcribe it to text using Aurelio's video-to-text endpoint.
2. Use Mistral LLMs to chat with our transcribed video content.
3. Add chat history to make our AI conversational.
4. Integrate async and streaming for a better UX and improved scalability.
5. See how we can optimize response latency and costs by reducing overall token count using semantic similarity, using Aurelio's chunking endpoint and Mistral's embedding models.

In [1]:
!pip install -qU \
  aurelio-sdk==0.0.18 \
  "yt-dlp[default]==2025.2.19" \
  mistralai==1.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.9 MB/s eta 0:00:00


In [2]:
!yt-dlp https://www.youtube.com/watch?v=JaHfCrVTYF4 -f mp4

[youtube] Extracting URL: https://www.youtube.com/watch?v=JaHfCrVTYF4
[youtube] JaHfCrVTYF4: Downloading webpage
[youtube] JaHfCrVTYF4: Downloading tv client config
[youtube] JaHfCrVTYF4: Downloading player f6e09c70
[youtube] JaHfCrVTYF4: Downloading tv player API JSON
[youtube] JaHfCrVTYF4: Downloading ios player API JSON
[youtube] JaHfCrVTYF4: Downloading m3u8 information
[info] JaHfCrVTYF4: Downloading 1 format(s): 18
[download] Destination: AI Agents as Neuro-Symbolic Systems？ [JaHfCrVTYF4].mp4
[download] 100% of   57.40MiB in 00:00:04 at 13.68MiB/s


We will use the [Aurelio Platform](https://platform.aurelio.ai/) for both video processing _and_ later for chunking. To follow the tutorial you can use the coupon `JBVIDEOAGENT` for free credits.

In [3]:
from aurelio_sdk import AurelioClient
import os
from getpass import getpass

os.environ["AURELIO_API_KEY"] = os.getenv("AURELIO_API_KEY") or getpass("Enter your Aurelio API key: ")

client = AurelioClient(api_key=os.environ["AURELIO_API_KEY"])

Now we send our video to Aurelio Platform for processing and chunking:

In [4]:
response_video_file = client.extract_file(
    file_path="/content/AI Agents as Neuro-Symbolic Systems？ [JaHfCrVTYF4].mp4",
    quality="low", chunk=False, wait=-1
)

We can access the transcribed video like so:

In [5]:
content = response_video_file.document.content
content

" Okay, so I wanted to put together a sort of overview video of what I'm currently working on, which is thinking or restructuring the way that I'm thinking about agents and the way that I'm also teaching or talking about agents. So this isn't going to be like a fully sort of edited and structured video. I just want to show you a little bit of what I'm thinking about and explain or explain where I'm coming from really.  So all in all, this is part of actually a broader thing that I am working on, which is actually why I haven't been posting on YouTube specifically for quite a while. And I think it's almost two months, which is the longest I think I haven't posted in forever. And, you know, it's well, okay, it's because I'm working on this, but it's also for other things as well. I had my first son like a month ago. So I've been pretty busy there. and just working on a lot of things over Aurelio as well. But I wanted to go through this introduction to  AI agents article that I'm working 

We can count the number of words from our transcribed video:

In [6]:
len(content.split())

4147

## Connecting an LLM

We're using Mistral AI in this example and we'll be using both their LLM and embed models, you can get [an API key from here](https://console.mistral.ai/api-keys).

In [7]:
import os
from mistralai import Mistral
from getpass import getpass

os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY") or \
    getpass("Enter your Mistral API key: ")

client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

In [8]:
from mistralai.models import (
    SystemMessage,
    UserMessage
)

response = client.chat.complete(
    model="mistral-large-latest",
    messages=[
        SystemMessage(content=(
            "You are an AI expert providing help to the user "
            "based on the content of the provided transcribed "
            "document.\n\n---\n\nTranscription:\n\n" +
            content
        )),
        UserMessage(content="Hi can you summarize this for me?")
    ]
)

We get our message content like so:

In [9]:
response.choices[0].message

AssistantMessage(content="Sure, here's a summarized version of the transcribed document:\n\nThe speaker discusses their evolving thoughts on AI agents and plans to create a structured video and course on this topic. They highlight that their recent absence from YouTube is due to this work and personal events.\n\n**Key Points:**\n\n1. **React Agent:**\n   - The React agent is a foundational structure for LM-based agents, allowing for multiple reasoning steps and tool calls.\n   - Example: Answering a question about the Apple Remote by using search tools and iterative reasoning.\n\n2. **Broad Definition of Agents:**\n   - The speaker argues that the common definition of agents (LM plus tool calls) is limiting.\n   - They prefer a neurosymbolic architecture, combining neural networks (like LLMs) with symbolic AI (handwritten rules, code).\n\n3. **Historical Context of AI:**\n   - **Symbolic AI:** Traditional approach using rules and ontologies (e.g., syllogistic logic).\n   - **Neural AI:

We can also track our usage:

In [10]:
response.usage

UsageInfo(prompt_tokens=5429, completion_tokens=435, total_tokens=5864)

## Adding Chat History

To make our video chat conversational we need to maintain chat history — to do that we'll write an `Agent` class that we can initialize and interact with, it will maintain our messages within this class.

In [11]:
from mistralai.models import AssistantMessage, UsageInfo

class Agent:
    messages: list[AssistantMessage | SystemMessage | UserMessage]
    usage: list[UsageInfo]

    def __init__(self):
        self.messages = [
            SystemMessage(content=(
                "You are an AI expert providing help to the user "
                "based on the content of the provided transcribed "
                "document.\n\n---\n\nTranscription:\n\n" +
                content
            ))
        ]
        self.usage = []

    def chat(self, content: str) -> AssistantMessage:
        # append user message to self.messages
        self.messages.append(UserMessage(content=content))
        # generate response
        response = client.chat.complete(
            model="mistral-large-latest",
            messages=self.messages
        )
        # append assistant message to self.messages
        self.messages.append(response.choices[0].message)
        # append usage (we can use this later)
        self.usage.append(response.usage)
        return response.choices[0].message

Now we can chat with our conversational history agent:

In [12]:
from IPython.display import display, Markdown

# initialize
agent = Agent()

res = agent.chat(
    content="can you summarize the meaning of 'symbolic' in this article?"
)
# print output in markdown
display(Markdown(res.content))

In the context of the article, "symbolic" refers to a traditional approach to artificial intelligence that involves using handwritten rules, ontologies, and logical functions to build AI systems. This approach is often associated with the early days of AI research, where the goal was to achieve true AGI (Artificial General Intelligence) through structured, logical frameworks.

Key points about the symbolic approach mentioned in the article include:

1. **Historical Context**: The symbolic approach was prominent in the 1940s, 1950s, and 1960s, and possibly into the 1970s. It is sometimes referred to as "good, old-fashioned AI" (GOFAI).

2. **Logical Frameworks**: This approach involves creating logical methodologies, such as syllogistic logic, where conclusions are drawn from major and minor premises. For example, if all dogs have four legs (major premise) and Japs is a dog (minor premise), then Japs has four legs (conclusion).

3. **Handwritten Rules**: The symbolic approach relies on written rules and philosophical grammars to construct deeper, AGI-type systems.

4. **Contrast with Neural AI**: Unlike neural or connectionist AI, which is inspired by the mechanisms of the brain and involves neural networks and perceptrons, the symbolic approach focuses on explicit, logical representations.

5. **Neurosymbolic Architecture**: The article discusses a neurosymbolic architecture, which combines neural networks (the neural part) with runnable code or handwritten rules (the symbolic part). This hybrid approach is seen as more flexible and powerful than using neural networks alone.

In summary, "symbolic" in this article refers to the use of explicit, logical rules and frameworks to build AI systems, contrasting with the neural approach that relies on learning and neural networks.

Now let's ask another question which requires context of the previous interactions:

In [13]:
res = agent.chat(
    content="can you give me that but in short bullet-points?"
)
# print output in markdown
display(Markdown(res.content))

Sure, here are the key points about "symbolic" in the article in short bullet-points:

- **Historical Context**: Prominent in the 1940s-1970s, often called "good, old-fashioned AI" (GOFAI).
- **Logical Frameworks**: Uses structured, logical methodologies like syllogistic logic.
- **Handwritten Rules**: Relies on explicit, handwritten rules and ontologies.
- **Contrast with Neural AI**: Unlike neural AI, which is inspired by brain mechanisms and uses neural networks.
- **Neurosymbolic Architecture**: Combines neural networks with runnable code or handwritten rules for a more flexible AI system.

These points summarize the meaning of "symbolic" in the context of the article.

## Async and Streaming

When developing AI apps that rely heavily on external APIs we tend to write async code to make our applications more scalable. With async code the time that our code would be spent waiting for API responses can instead be spent performing other tasks.

We will rewrite our `Agent` class to work fully asynchronously, and we'll also add streaming — which can provide an improved user experience as we can show the user the tokens as soon as they're generated.

In [14]:
import asyncio


class Agent:
    messages: list[AssistantMessage | SystemMessage | UserMessage]
    usage: list[UsageInfo]
    queue: asyncio.Queue | None = None

    def __init__(self):
        self.messages = [
            SystemMessage(content=(
                "You are an AI expert providing help to the user "
                "based on the content of the provided transcribed "
                "document.\n\n---\n\nTranscription:\n\n" +
                content
            ))
        ]
        self.usage = []

    async def chat(self, content: str) -> AssistantMessage:
        # append user message to self.messages
        self.messages.append(UserMessage(content=content))
        # generate response asynchronously
        response = await client.chat.stream_async(
            model="mistral-large-latest",
            messages=self.messages
        )
        # full response object to be built
        all_tokens = []
        all_usage = []
        # iterate through the token generator and add to queue
        async for chunk in response:
            if (token := chunk.data.choices[0].delta.content) is not None:
                print(token, end="", flush=True)
                all_tokens.append(token)
        # append assistant message to self.messages
        self.messages.append(AssistantMessage(content="".join(all_tokens)))
        # append usage (we can use this later)
        self.usage.append(chunk.data.usage)
        return self.messages[-1]

In [15]:
agent = Agent()

res = await agent.chat(
    content="can you summarize the meaning of 'symbolic' in this article?"
)

In the context of the article, "symbolic" refers to the traditional approach to artificial intelligence that relies on handwritten rules, ontologies, and logical functions to build AI systems. This approach is often associated with the early days of AI research, where the goal was to achieve true Artificial General Intelligence (AGI) through structured, logical frameworks.

Key points about the symbolic approach mentioned in the article:

1. **Historical Context**: The symbolic approach was prominent in the 1940s, 1950s, and 1960s, and possibly into the 1970s. It is sometimes referred to as "good old-fashioned AI" (GOFAI).

2. **Logical Frameworks**: This approach involves using logical frameworks like syllogistic logic, which includes major premises, minor premises, and conclusions. For example, if all dogs have four legs and Japs is a dog, then Japs has four legs.

3. **Handwritten Rules**: Symbolic AI relies on manually crafted rules and ontologies to create intelligent systems. Thi

We can continue our conversation:

In [16]:
res = await agent.chat(
    content="tell me in more detail what was said on point (1)"
)

Certainly! Here's a more detailed breakdown of point (1), the historical context of the symbolic approach to AI as discussed in the article:

### Historical Context of Symbolic AI

**Early Days of AI Research:**
- The symbolic approach to AI has its roots in the early days of AI research, roughly from the 1940s to the 1970s.
- During this period, AI researchers believed that true Artificial General Intelligence (AGI) could be achieved through the creation of explicit rules, ontologies, and logical functions.

**Philosophical Foundations:**
- The philosophical underpinnings of symbolic AI can be traced back to logical frameworks like syllogistic logic, which was developed by Aristotle.
- Syllogistic logic involves using major premises, minor premises, and conclusions to form logical deductions. For example:
  - Major Premise: All dogs have four legs.
  - Minor Premise: Japs is a dog.
  - Conclusion: Japs has four legs.

**Good Old-Fashioned AI (GOFAI):**
- The symbolic approach is somet

In [17]:
res = await agent.chat(
    content="and what does that have to do with AI agents?"
)

In the context of the article, the historical and philosophical foundations of symbolic AI are relevant to the discussion on AI agents in several ways:

### Broadening the Definition of AI Agents

1. **Neuro-Symbolic Architecture**:
   - The article introduces the concept of a neuro-symbolic architecture, which combines neural networks (the neural part) with handwritten code or rules (the symbolic part). This hybrid approach is presented as a more comprehensive definition of AI agents.
   - By incorporating both neural and symbolic elements, AI agents can leverage the strengths of traditional rule-based systems and modern data-driven learning methods.

2. **Flexibility and Versatility**:
   - The author argues that limiting the definition of AI agents to just Large Language Models (LLMs) that can call tools is too restrictive. Instead, AI agents should be seen as systems that can incorporate a variety of neural network-based models and symbolic components.
   - This broader definition 

## Optimizing Token Cost

Throwing the full article into each interaction will give us maximal accuracy but is also _expensive_.

We can see this by checking our token usage and calculating our costs. To check our usage we simply access our agent's `usage` attribute:

In [18]:
agent.usage

[UsageInfo(prompt_tokens=5435, completion_tokens=369, total_tokens=5804),
 UsageInfo(prompt_tokens=5820, completion_tokens=613, total_tokens=6433),
 UsageInfo(prompt_tokens=6447, completion_tokens=729, total_tokens=7176)]

To estimate our costs for running our agent, we can take the latest pricing for Mistral's large model for `prompt_tokens` (input) and `completion_tokens` (output) from their [pricing page](https://mistral.ai/en/products/la-plateforme#pricing).

As of 08 March 2025 those prices are:

| Model | API name | Input (/M tokens) | Output (/M tokens) |
| ----- | -------- | ----------------- | ------------------ |
| Mistral Large 24.11 | mistral-large-latest | \$2 | \$6 |

We can define a cost calculator like so:

In [19]:
def cost(usage: UsageInfo) -> float:
    input_cost = usage.prompt_tokens * 2e-6
    output_cost = usage.completion_tokens * 2e-6
    return round(input_cost + output_cost, 5)

In [20]:
# save these values for later reference
original_cost = []

for usage in agent.usage:
    usage_cost = cost(usage=usage)
    original_cost.append(usage_cost)
    print(f"${usage_cost}")

$0.01161
$0.01287
$0.01435


These seem like small numbers but they will quickly add up as we continue throwing the full transcription into our LLM with every new interaction.

To optimize our token cost we can pull in _only_ the most relevant chunks of information, making use of semantic similarity. To do this we will:

1. Break our transcribed document into smaller chunks.
2. Embed those chunks into vector embeddings.
3. Store those vector embeddings in a numpy array.
4. When querying, our LLM (now agent) will transform our question into a small query.
5. We embed that query into a vector embedding.
6. Compare the semantic similarity between our query vector and our chunk vectors to find the most similar chunks.
7. Return those chunks to our LLM ready for a final response.

Let's start by chunking our document, we use the _async_ Aurelio chunking endpoint for this:

In [21]:
from aurelio_sdk import AsyncAurelioClient

# we reinitialize the client for async
aurelio_client = AsyncAurelioClient()

Call the chunk endpoint on our document:

In [22]:
from aurelio_sdk import ChunkingOptions

# we use a semantic chunker with a max chunk length of 400 tokens
chunking_options = ChunkingOptions(
    chunker_type="semantic",
    max_chunk_length=500,
    window_size=3
)

chunks = await aurelio_client.chunk(
    content=content,
    processing_options=chunking_options
)

We can see the chunks like so:

In [23]:
chunks.document.chunks[5:10]

[ResponseChunk(id='chunk_7427b07a-7203-4778-8be9-2698176507f9', content="I know that Apple remote controls the front row program.  but what other device controls the front row program. So it says, okay, based on this, my next reasoning step is I need to search front row and find other devices that control it. So then it does this search for front row. It could also probably do something, if we're thinking in rag terms here, it could be like device to control front row. And probably more today LM would do that. But that's fine. this is just an example. So it goes back to the search tool again and it says query.", chunk_index=6, num_tokens=121, metadata={}),
 ResponseChunk(id='chunk_8c3acf5f-33b8-419c-94e5-4dbf706b1496', content="Front row. And this isn't  I've shortened this for the sake of brevity. I think in the actual example, or at least from the paper, the actual example returns a lot more information. But this is the part of it that is important.", chunk_index=7, num_tokens=54, me

Next, we need to embed each chunk. For that we will use Mistral's embedding model.

In [24]:
embeddings_response = await client.embeddings.create_async(
    model="mistral-embed",
    inputs=[x.content for x in chunks.document.chunks]
)

This returns a list of `EmbeddingResponseData` objects inside the `.data` attribute:

In [25]:
len(embeddings_response.data)

35

Each of those `EmbeddingResponseData` objects contains a single vector embedding inside the `.embedding` attribute. The dimensionality of the embedding model is `1024` which we can see below:

In [26]:
len(embeddings_response.data[0].embedding)

1024

Next we must add all of these vectors to a single numpy array:

In [27]:
import numpy as np

video_emb = np.asarray([x.embedding for x in embeddings_response.data])
# check the dimensionality of our video chunks array
video_emb.shape

(35, 1024)

From this we can see we have 35 1024-dimensional vector embeddings that represent our full transcribed document. Now to plug this back into our agent we will create a `query` tool that will allow our LLM to provide a natural language query and return the most relevant chunks based on that query.

Let's start by writing this tool step-by-step. First, we create a _query vector_:

In [28]:
query = "what is the relationship between AI agents and GOFAI?"

embeddings_response = await client.embeddings.create_async(
    model="mistral-embed",
    inputs=[query]
)
xq = np.asarray(embeddings_response.data[0].embedding)
xq.shape

(1024,)

Now calculate the dot product similarity between our query vector `xq` and precomputed document chunk vectors.

In [29]:
sim_arr = np.dot(xq, video_emb.T)
sim_arr

array([0.6828565 , 0.75058198, 0.69919688, 0.67968833, 0.61341964,
       0.60867901, 0.62188296, 0.60628245, 0.60307181, 0.74770941,
       0.76357249, 0.61461822, 0.75837007, 0.75699921, 0.64369794,
       0.71479521, 0.65117063, 0.68480309, 0.71202544, 0.69951878,
       0.68277037, 0.68066402, 0.74412483, 0.64809726, 0.66284248,
       0.64783145, 0.62315144, 0.62955026, 0.7012869 , 0.70282584,
       0.67379309, 0.68034123, 0.70010112, 0.61695457, 0.57207087])

Now we return the indexes for the `top_k` most similar (highest scoring) chunks:

In [30]:
top_k = 3  # we'll set top_k to 3, returning the 3 most similar chunks

most_similar_idx = np.argsort(sim_arr)[-top_k:][::-1]
most_similar_idx

array([10, 12, 13])

Before pulling the content of each chunk, we convert our list of chunks into an array of chunks — these will speed up our chunk content retrieval later.

In [31]:
chunks_content = np.asarray([x.content for x in chunks.document.chunks])
chunks_content.shape

(35,)

Now retrieve the chunks content:

In [32]:
chunks_content[most_similar_idx]

array([". .  basically agent, LM agent. I think this came just before the React agent paper. It's very similar, I would say, has a bit less structured in the React agent. But, yeah, it's super relevant. And the way that they described their system was that it was a neurosymbolic architecture. I really like this definition because a, so neurosymbolic architecture, It's two things, right? You have the neural part, you have the symbolic part. And I actually have another kind of starting  on this article but it's uh yeah there's this mostly notes at the moment so the neural part of this in fact let's start with the symbolic part the symbolic part is the more traditional AI right so the you know I think this is back in the 40s 50s 60s mostly and then maybe so actually 70s as well this was actually maybe not 70s this was this was a the sort of traditional approach to AI. And the idea,  or the symbolists that were just like full on symbolists felt that true AGI would be achieved through writt

We can see here that we've returned the chunks of our article _most_ relevant to our query. Now we wrap all of this up into a single function, ie our _tool_:

In [33]:
async def search(query: str) -> str:
    """Use this tool to search for relevant chunks of information
    from the provided video. Provide as much context as possible
    to the `query` parameter, ensuring to write your search
    query in natural language. If you must answer multiple
    questions you should use this tool to only answer one at a
    time. Do not include multiple questions in the `query`."""
    # embed our query to create a 'query vector'
    embeddings_response = await client.embeddings.create_async(
        model="mistral-embed",
        inputs=[query]
    )
    xq = np.asarray(embeddings_response.data[0].embedding)
    # perform the similarity search
    sim_arr = np.dot(xq, video_emb.T)
    # get the top_k most similar chunks
    most_similar_idx = np.argsort(sim_arr)[-top_k:][::-1]
    # return our most relevant chunks
    return "\n---\n".join(chunks_content[most_similar_idx].tolist())

Let's test quickly:

In [34]:
print(await search(query=query))

. .  basically agent, LM agent. I think this came just before the React agent paper. It's very similar, I would say, has a bit less structured in the React agent. But, yeah, it's super relevant. And the way that they described their system was that it was a neurosymbolic architecture. I really like this definition because a, so neurosymbolic architecture, It's two things, right? You have the neural part, you have the symbolic part. And I actually have another kind of starting  on this article but it's uh yeah there's this mostly notes at the moment so the neural part of this in fact let's start with the symbolic part the symbolic part is the more traditional AI right so the you know I think this is back in the 40s 50s 60s mostly and then maybe so actually 70s as well this was actually maybe not 70s this was this was a the sort of traditional approach to AI. And the idea,  or the symbolists that were just like full on symbolists felt that true AGI would be achieved through written rules

Now we need to redefine our `Agent` and plug our new tool into it. To do that we need to format our tool so that the Mistral API can read it:

In [35]:
import inspect

inspect.getdoc(search)

'Use this tool to search for relevant chunks of information\nfrom the provided video. Provide as much context as possible\nto the `query` parameter, ensuring to write your search\nquery in natural language. If you must answer multiple \nquestions you should use this tool to only answer one at a\ntime. Do not include multiple questions in the `query`.'

In [36]:
func_schema = {
    "name": search.__name__,
    "description": inspect.getdoc(search),
    "parameters": {"type": "object", "properties": {}, "required": []}
}
func_schema

{'name': 'search',
 'description': 'Use this tool to search for relevant chunks of information\nfrom the provided video. Provide as much context as possible\nto the `query` parameter, ensuring to write your search\nquery in natural language. If you must answer multiple \nquestions you should use this tool to only answer one at a\ntime. Do not include multiple questions in the `query`.',
 'parameters': {'type': 'object', 'properties': {}, 'required': []}}

In [37]:
dtype_map = {
    int: "number",
    float: "number",
    str: "string",
    bool: "boolean",
    None: "null",
    list: "array",
}

In [38]:
signature = inspect.signature(search)
for name, dtype in signature.parameters.items():
    # add param to properties
    func_schema["parameters"]["properties"][name] = {
        "type": dtype_map.get(dtype.annotation, "object")
    }
    # and required (assuming all are required)
    func_schema["parameters"]["required"].append(name)

We now have our fully defined function schema:

In [39]:
func_schema

{'name': 'search',
 'description': 'Use this tool to search for relevant chunks of information\nfrom the provided video. Provide as much context as possible\nto the `query` parameter, ensuring to write your search\nquery in natural language. If you must answer multiple \nquestions you should use this tool to only answer one at a\ntime. Do not include multiple questions in the `query`.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

We transform this into a `mistralai` `Function` object:

In [40]:
from mistralai.models.function import Function

tool_signatures = [
    {
        "type": "function",
        "function": Function(
            name=func_schema["name"],
            description=func_schema["description"],
            parameters=func_schema["parameters"]
        )
    }
]

We can add our new `tool_signatures` list to our completion call within our `Agent`.

In [41]:
import asyncio
from typing import Callable


class Agent:
    messages: list[AssistantMessage | SystemMessage | UserMessage]
    usage: list[UsageInfo]
    tool_signatures: list[Function]
    queue: asyncio.Queue | None = None

    def __init__(self, tool_signatures: list[Function]):
        self.messages = [
            SystemMessage(content=(
                "You are an AI expert providing help to the user "
                "based on the content of the provided transcribed "
                "document."
            ))
        ]
        self.usage = []
        self.tool_signatures = tool_signatures

    async def chat(self, content: str) -> AssistantMessage:
        # append user message to self.messages
        self.messages.append(UserMessage(content=content))
        # generate response asynchronously
        response = await client.chat.stream_async(
            model="mistral-large-latest",
            messages=self.messages,
            tools=self.tool_signatures,
            tool_choice="auto"
        )
        # full response object to be built
        all_tokens = []
        all_usage = []
        # iterate through the token generator and add to queue
        async for chunk in response:
            if isinstance((tool_call := chunk.data.choices[0].delta.tool_calls), list):
                print(tool_call)
            elif (token := chunk.data.choices[0].delta.content) is not None:
                print(token, end="", flush=True)
                all_tokens.append(token)
        # append assistant message to self.messages
        #self.messages.append(AssistantMessage(content="".join(all_tokens)))
        # append usage (we can use this later)
        #self.usage.append(chunk.data.usage)
        return self.messages[-1], tool_call

In [42]:
agent = Agent(tool_signatures=tool_signatures)

res = await agent.chat(
    content="can you summarize the meaning of 'symbolic' in this article?"
)

[ToolCall(function=FunctionCall(name='search', arguments='{"query": "Meaning of \'symbolic\' in the provided article"}'), id='sKGMXCUCQ', type=None, index=0)]


Our video agent can now create a tool call but it cannot execute the tool call — for that we need a little more scaffolding to handle the detection of a tool call coming from our LLM and the translation of that into execution of our `search` function.

To do that, we will create a tool execution function:

In [43]:
import json
from mistralai.models import ToolCall, ToolMessage

tools = [search]

tool_map = {t.__name__: t for t in tools}

async def execute_tool(tool_call: ToolCall) -> ToolMessage:
    tool_name = tool_call.function.name
    tool_params = json.loads(tool_call.function.arguments)
    tool_call_id = tool_call.id
    out = await tool_map[tool_name](**tool_params)
    return ToolMessage(
        content=out,
        name=tool_name,
        tool_call_id=tool_call_id
    )

Now let's take the `tool_call` from our previous `Agent.chat` call and run it through our `execute_tool` function.

In [44]:
res[1][0]

ToolCall(function=FunctionCall(name='search', arguments='{"query": "Meaning of \'symbolic\' in the provided article"}'), id='sKGMXCUCQ', type=None, index=0)

In [45]:
tool_message = await execute_tool(tool_call=res[1][0])
tool_message

ToolMessage(content=". .  basically agent, LM agent. I think this came just before the React agent paper. It's very similar, I would say, has a bit less structured in the React agent. But, yeah, it's super relevant. And the way that they described their system was that it was a neurosymbolic architecture. I really like this definition because a, so neurosymbolic architecture, It's two things, right? You have the neural part, you have the symbolic part. And I actually have another kind of starting  on this article but it's uh yeah there's this mostly notes at the moment so the neural part of this in fact let's start with the symbolic part the symbolic part is the more traditional AI right so the you know I think this is back in the 40s 50s 60s mostly and then maybe so actually 70s as well this was actually maybe not 70s this was this was a the sort of traditional approach to AI. And the idea,  or the symbolists that were just like full on symbolists felt that true AGI would be achieved 

This is our executed tool output. We'd append this alongside an `AssistantMessage` for the initial LLM-generated tool call to our `Agent.messages` attribute, then feed everything back into our LLM for it to decide what to do next. Hopefully, we'll see our LLM deciding to use the information it gathered to respond to the user.

In [46]:
agent.messages.extend([
    AssistantMessage(content="", tool_calls=res[1]),
    tool_message
])
agent.messages

[SystemMessage(content='You are an AI expert providing help to the user based on the content of the provided transcribed document.', role='system'),
 UserMessage(content="can you summarize the meaning of 'symbolic' in this article?", role='user'),
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='search', arguments='{"query": "Meaning of \'symbolic\' in the provided article"}'), id='sKGMXCUCQ', type=None, index=0)], prefix=False, role='assistant'),
 ToolMessage(content=". .  basically agent, LM agent. I think this came just before the React agent paper. It's very similar, I would say, has a bit less structured in the React agent. But, yeah, it's super relevant. And the way that they described their system was that it was a neurosymbolic architecture. I really like this definition because a, so neurosymbolic architecture, It's two things, right? You have the neural part, you have the symbolic part. And I actually have another kind of starting  on this articl

In [47]:
# generate response asynchronously
response = await client.chat.stream_async(
    model="mistral-large-latest",
    messages=agent.messages,
    tools=agent.tool_signatures,
    tool_choice="auto"
)
# full response object to be built
all_tokens = []
all_usage = []
# iterate through the token generator and add to queue
async for chunk in response:
    if isinstance((tool_call := chunk.data.choices[0].delta.tool_calls), list):
        print(tool_call)
    elif (token := chunk.data.choices[0].delta.content) is not None:
        print(token, end="", flush=True)
        all_tokens.append(token)

In the article, the term 'symbolic' refers to the traditional approach to artificial intelligence (AI) that was prevalent from the 1940s to the 1960s and possibly into the 1970s. This approach is characterized by the use of written rules, ontologies, and logical functions to achieve true artificial general intelligence (AGI). The symbolists believed that AGI could be attained through these methods, which involved creating philosophical grammars and logical frameworks.

An example of symbolic logic is Aristotle's syllogistic logic, which involves a major premise, a minor premise, and a conclusion derived from these premises. For instance, a syllogism might state that all dogs have four legs (major premise), and if a specific animal is a dog (minor premise), then that animal has four legs (conclusion).

The article also discusses how neural networks, which are a key component of modern AI, learn logical representations of concepts, somewhat akin to symbols, but these are not handwritten.

After adding these additional tool call messages our LLM is able to respond to our query directly. Now let's integrate all of this back into a new `Agent` class.

In [48]:
class Agent:
    messages: list[AssistantMessage | SystemMessage | UserMessage]
    usage: list[UsageInfo]
    tool_signatures: list[Function]
    queue: asyncio.Queue | None = None

    def __init__(self, tool_signatures: list[Function], max_steps: int = 3):
        self.messages = [
            SystemMessage(content=(
                "You are an AI expert providing help to the user "
                "based on the content of the provided transcribed "
                "document."
            ))
        ]
        self.usage = []
        self.tool_signatures = tool_signatures
        self.max_steps = max_steps

    async def chat(self, content: str) -> AssistantMessage:
        # append user message to self.messages
        self.messages.append(UserMessage(content=content))
        # we will need to enter a loop to support multiple iterations
        step = 0
        while step <= self.max_steps:
            # generate response asynchronously
            response = await client.chat.stream_async(
                model="mistral-large-latest",
                messages=self.messages,
                tools=self.tool_signatures,
                tool_choice="auto"
            )
            # full response object to be built
            all_tokens = []
            all_usage = []
            # iterate through the token generator and add to queue
            async for chunk in response:
                if isinstance(
                    (tool_calls := chunk.data.choices[0].delta.tool_calls), list
                ):
                    # print the tool call in a cleaner format
                    print(
                        f"{tool_calls[0].function.name}: "
                        f"{tool_calls[0].function.arguments}"
                    )
                    # we execute our tool
                    tool_message = await execute_tool(tool_call=tool_calls[0])
                    # and add the assistant tool call and tool output message
                    # to our self.messages
                    self.messages.extend([
                        AssistantMessage(content="", tool_calls=tool_calls),
                        tool_message
                    ])

                elif (token := chunk.data.choices[0].delta.content) is not None:
                    print(token, end="", flush=True)
                    all_tokens.append(token)
            # append usage (we can use this later)
            self.usage.append(chunk.data.usage)
            # append assistant message to self.messages (if returned)
            if len(all_tokens) > 1:
                self.messages.append(AssistantMessage(content="".join(all_tokens)))
                break
            step += 1
        return self.messages[-1], tool_call

Now let's try another query, this time we will try to allow our agent to use the `search` tool twice to collate information from various chunks.

In [49]:
agent = Agent(tool_signatures=tool_signatures)

res = await agent.chat(
    content=(
        "Does the document mention 'good old fashioned AI'? And does it "
        "say anything about deepseek? How does the document "
        "compare the two?"
    )
)

search: {"query": "good old fashioned AI"}
Yes, the document does mention the term 'good old fashioned AI.' This term is used to refer to traditional AI approaches, also known as symbolic AI. These methods involve using logical frameworks and symbolic reasoning to build AI systems. The document discusses how this traditional approach aimed to achieve AI through written rules, ontologies, and logical functions.

Next, I'll look for mentions of 'deepseek' in the document.search: {"query": "deepseek"}


Despite performing two searches to answer this query, we still used _significantly_ less tokens:

In [50]:
agent.usage

[UsageInfo(prompt_tokens=187, completion_tokens=23, total_tokens=210),
 UsageInfo(prompt_tokens=897, completion_tokens=112, total_tokens=1009)]

Without chunking we spent ~$0.12 on a single query (with a single question):

In [53]:
original_cost

[0.01161, 0.01287, 0.01435]

With chunking, for two questions within a single query we're spending:

In [51]:
for usage in agent.usage:
    print(f"${cost(usage=usage)}")

$0.00042
$0.00202


A total of ~$0.0024, a dramatic six-fold reduction in price.

---

With that we've built an agent capable of helping us understand videos. We've also taken steps to drastically optimize expenditure.